# Hair Detector

#### Created by: Matheus Gabriel

Project to detect type of hair using one deep learning model.


## Imports

In [2]:
import numpy as np
import tensorflow as tf
import os
import cv2
from keras.preprocessing import image
from os.path import isfile, join


## Load images to train and test the model

In [84]:
train_datagen = image.ImageDataGenerator(rescale = 1./255, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True )
train_set = train_datagen.flow_from_directory('data/Train/',
                                                 target_size = (80, 80),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 553 images belonging to 3 classes.


In [85]:
test_datagen = image.ImageDataGenerator(rescale = 1./255, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True )
test_set = test_datagen.flow_from_directory('data/Test/',
                                                 target_size = (80, 80),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 218 images belonging to 3 classes.


## Defining the layers of model

In [86]:
cnn = tf.keras.models.Sequential()

cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[80, 80, 3]))

cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))

cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

cnn.add(tf.keras.layers.Dropout(0.3))

cnn.add(tf.keras.layers.Flatten())

cnn.add(tf.keras.layers.Dense(units=128, activation=tf.keras.layers.LeakyReLU()))

cnn.add(tf.keras.layers.Dropout(0.3))

cnn.add(tf.keras.layers.Dense(units=64, activation='relu'))

cnn.add(tf.keras.layers.Dropout(0.2))

cnn.add(tf.keras.layers.Dense(units=3, activation='softmax'))

## Training the model 

In [87]:
cnn.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

cnn.fit(x = train_set, validation_data = test_set, epochs = 30)

Epoch 1/30
18/18 [==============================] - 8s 443ms/step - loss: 1.0758 - accuracy: 0.4485 - val_loss: 1.1185 - val_accuracy: 0.4174
Epoch 2/30
18/18 [==============================] - 7s 412ms/step - loss: 0.9932 - accuracy: 0.5371 - val_loss: 1.1233 - val_accuracy: 0.3440
Epoch 3/30
18/18 [==============================] - 7s 406ms/step - loss: 0.9405 - accuracy: 0.5642 - val_loss: 1.0478 - val_accuracy: 0.5000
Epoch 4/30
18/18 [==============================] - 7s 415ms/step - loss: 0.8892 - accuracy: 0.6022 - val_loss: 1.0107 - val_accuracy: 0.4862
Epoch 5/30
18/18 [==============================] - 7s 410ms/step - loss: 0.8065 - accuracy: 0.6329 - val_loss: 0.9052 - val_accuracy: 0.5734
Epoch 6/30
18/18 [==============================] - 7s 412ms/step - loss: 0.8503 - accuracy: 0.6112 - val_loss: 1.0075 - val_accuracy: 0.4725
Epoch 7/30
18/18 [==============================] - 7s 405ms/step - loss: 0.8101 - accuracy: 0.6600 - val_loss: 0.9549 - val_accuracy: 0.5183
Epoch 

## Function to return the type of hair

In [89]:
def predictCabelo(img):
    test_image = image.load_img(img, target_size = (80, 80))
    test_image = image.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis = 0)
    result = cnn.predict(test_image)

    if(result[0][0] == max(result[0])):
        return 'cacheado'
    
    if(result[0][1] == max(result[0])):
        return 'liso'

    if(result[0][2] == max(result[0])):
        return'ondulado'

## Load `advertising.mp4` to extract frames and generate the predictions

In [93]:
vidcap = cv2.VideoCapture('advertising.mp4')

def getFrame(sec):
    vidcap.set(cv2.CAP_PROP_POS_MSEC,sec*1000)
    hasFrames,image = vidcap.read()
    if hasFrames:
        cv2.imwrite(f"frames/{count}.jpg", image)
        prediction = predictCabelo(f"frames/{count}.jpg")
        cv2.rectangle(image,(150,15),(500,85),(255,255,255), -1)
        cv2.rectangle(image,(150,15),(500,85),(56,142,72), 2)
        cv2.putText(image,prediction,(0 + 200,0 + 0 + 50), cv2.FONT_HERSHEY_COMPLEX,1, (56,142,72), 2, cv2.LINE_AA)  
        cv2.imwrite(f"frames/{count}.jpg", image)
    return hasFrames

sec = 0
frameRate = 0.5
count=1
success = getFrame(sec)
while success:
    count = count + 1
    sec = sec + frameRate
    sec = round(sec, 2)
    success = getFrame(sec)

## Join the frames in `video.avi`

In [94]:
pathIn= './frames/'
pathOut = 'video.avi'
fps = 0.5
frame_array = []
files = [f for f in os.listdir(pathIn) if isfile(join(pathIn, f))]

files.sort(key = lambda x: x[5:-4])
files.sort()
frame_array = []
files = [f for f in os.listdir(pathIn) if isfile(join(pathIn, f))]

files.sort(key = lambda x: x[5:-4])
for i in range(len(files)):
    filename=pathIn + files[i]
    img = cv2.imread(filename)
    height, width, layers = img.shape
    size = (width,height)
    frame_array.append(img)
out = cv2.VideoWriter(pathOut,cv2.VideoWriter_fourcc(*'DIVX'), fps, size)
for i in range(len(frame_array)):
    # writing to a image array
    out.write(frame_array[i])
out.release()